In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
import os
from collections import defaultdict
from getpass import getpass
from copy import deepcopy
import pickle
import pandas as pd

In [ ]:
if 'HELICONE_API_KEY' not in os.environ:
    print("You didn't set your Helicone key to the HELICONE_API_KEY env var on the command line.")
    os.environ['HELICONE_API_KEY'] = getpass("Please enter your Helicone API Key now: ")

You didn't set your Helicone key to the HELICONE_API_KEY env var on the command line.
Please enter your Helicone API Key now: ··········


In [ ]:
client = OpenAI(base_url="https://oai.hconeai.com/v1", api_key=os.environ['HELICONE_API_KEY'])

In [ ]:
def create_prompts_and_poems(n, poem_frequency_penalty, temperature, instruction, output_filename = "poem_dataset.pkl"):
  prompt = f"Give me {n} unique prompts to generate a poem with. Output only the prompts seperated by a semi-colon. each prompt should be in the form of: Write me a poem about X, in the style of Y, in the structure of Z.; Where X is a topic, Y is a style of a person and Z is a set poem strucutre (e.g. sonnet, haiku, etc). Don't use a semi-colon anywhere else except to seperate prompts."
  messages = [
      {'role': 'system', 'content': prompt}
  ]

  client = OpenAI(base_url="https://oai.hconeai.com/v1", api_key=os.environ['HELICONE_API_KEY'])
  print(f"Generating {n} poem prompts with poem frequency difficulty {poem_frequency_penalty}")

  response = client.chat.completions.create(
      model='gpt-4',
      messages=messages,
      temperature=1,
      max_tokens=8000,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
  )
  gpt_response = response.choices[0].message.content
  poem_list = gpt_response.split("; ")
  print(f"Length of poem list: {len(poem_list)}")

  poem_map = {}
  count = 1
  for poem_prompt in poem_list:
    print(f"Generating poem: {count}")
    messages2 = [
        {'role': 'system', 'content': poem_prompt + ". Limit the poem to be 40 words." + instruction}
    ]

    response2 = client.chat.completions.create(
        model='gpt-4',
        messages=messages2,
        temperature=temperature,
        max_tokens=100,
        top_p=1.0,
        frequency_penalty=poem_frequency_penalty,
        presence_penalty=0
    )

    poem_response = response2.choices[0].message.content
    poem_map[poem_prompt] = poem_response
    count += 1

  poems_df = pd.DataFrame(list(poem_map.items()), columns=['Poem Prompt', 'Poem'])
  return poems_df

In [ ]:
easy_poems_df = create_prompts_and_poems(75, poem_frequency_penalty=0, temperature=0.5, instruction="")
medium_poems_df = create_prompts_and_poems(75, poem_frequency_penalty=0.5, temperature=1, instruction=" Talk about the topic of the poem less obviously.")
hard_poems_df = create_prompts_and_poems(75, poem_frequency_penalty=0.7, temperature=1.5, instruction=" Do not mention the topic of the poem explicity.")

Generating 75 poem prompts with poem frequency difficulty 0
Length of poem list: 71
Generating poem: 1
Generating poem: 2
Generating poem: 3
Generating poem: 4
Generating poem: 5
Generating poem: 6
Generating poem: 7
Generating poem: 8
Generating poem: 9
Generating poem: 10
Generating poem: 11
Generating poem: 12
Generating poem: 13
Generating poem: 14
Generating poem: 15
Generating poem: 16
Generating poem: 17
Generating poem: 18
Generating poem: 19
Generating poem: 20
Generating poem: 21
Generating poem: 22
Generating poem: 23
Generating poem: 24
Generating poem: 25
Generating poem: 26
Generating poem: 27
Generating poem: 28
Generating poem: 29
Generating poem: 30
Generating poem: 31
Generating poem: 32
Generating poem: 33
Generating poem: 34
Generating poem: 35
Generating poem: 36
Generating poem: 37
Generating poem: 38
Generating poem: 39
Generating poem: 40
Generating poem: 41
Generating poem: 42
Generating poem: 43
Generating poem: 44
Generating poem: 45
Generating poem: 46
Gener

In [ ]:
import re

# Function to extract topic, style, and structure from a text string
def extract_parts(text):
    pattern = r"about (?P<topic>.*), in the style of (?P<style>.*), in the structure of (?P<structure>.*)."
    match = re.search(pattern, text)
    if match:
        return match.groupdict()
    else:
        return {'topic': None, 'style': None, 'structure': None}

# Function to apply the extraction to each row in the DataFrame
def apply_extraction_to_df(df):
    extracted_data = df['Poem Prompt'].apply(extract_parts)
    df['Topic'] = extracted_data.apply(lambda x: x.get('topic'))
    df['Style'] = extracted_data.apply(lambda x: x.get('style'))
    df['Structure'] = extracted_data.apply(lambda x: x.get('structure'))
    return df

In [ ]:
easy_poems_compiled_df = compiled_df = apply_extraction_to_df(easy_poems_df)
medium_poems_compiled_df = compiled_df = apply_extraction_to_df(medium_poems_df)
hard_poems_compiled_df = compiled_df = apply_extraction_to_df(hard_poems_df)

In [ ]:
easy_poems_compiled_df

,Poem Prompt,Poem,Topic,Style,Structure
0,"Write me a poem about the sunrise, in the styl...","Sun's hymn whispers in the morning's ear,\nIn ...",the sunrise,Maya Angelou,a sonne
1,"Write me a poem about first love, in the style...","In the garden of youth, first love blooms brig...",first love,Rumi,a ghaza
2,"Write me a poem about war, in the style of Wil...","Through trenches deep, where brave hearts slee...",war,Wilfred Owen,a balla
3,"Write me a poem about the ocean, in the style ...","In depths of blue, secrets lie,\nA realm where...",the ocean,Lord Byron,a haik
4,"Write me a poem about winter, in the style of ...","In winter's grasp, the world lies still,\nUnde...",winter,Robert Frost,a sestin
...,...,...,...,...,...
66,"Write me a poem about starlight, in the style ...","Starlight's grace on the meadow's face,\nLike ...",starlight,Thomas Hardy,a triole
67,"Write me a poem about dusk, in the style of Em...","Dusk descends, a soft embrace,\nAs day yields ...",dusk,Emily Bronte,a ronde
68,"Write me a poem about a castle, in the style o...","In shadows deep, a castle bleak,\nBeneath a mo...",a castle,Edgar Allan Poe,a sestin
69,"Write me a poem about a cowboy, in the style o...","Under the vast, star-kissed prairie night,\nA ...",a cowboy,Louis L Amour,a balla


In [ ]:
medium_poems_compiled_df

,Poem Prompt,Poem,Topic,Style,Structure
0,"Write me a poem about a blooming garden, in th...","A Vision of Heaven’s dainty Eye,\nIn lured eme...",a blooming garden,Emily Dickinson,a sonne
1,"Write me a poem about the vastness of space, i...","Beyond the humdrum troubles in our caste,\nIn ...",the vastness of space,W.H. Auden,an od
2,Write me a poem about the complexities of love...,"Shadowed lane in fall,\nLeaves twine like trus...",the complexities of love,Robert Frost,a haik
3,"Write me a poem about sunrise, in the style of...","In brisk morning’s silent sanctity, all aglow,...",sunrise,Walt Whitman,a villanell
4,"Write me a poem about the vibrant city life, i...","In veins of steel, pulses light's marquee,\nSt...",the vibrant city life,Langston Hughes,a sestin
...,...,...,...,...,...
60,"Write me a poem about the wonder of art, in th...","In stained mosaic dreams, whispered in colors ...",the wonder of art,Picasso,an od
61,"\n\nWrite me a poem about the endless desert, ...","In amber waves reside tales of yore,\nFierce w...",the endless desert,Lawrence of Arabia,a pantou
62,"Write me a poem about the call of the wild, in...","In whispers of pine, stark echoes rebound,\nBe...",the call of the wild,Jack London,a sonne
63,"Write me a poem about an unlikely friendship, ...","Autumn leaves afloat,\nFox and crow share flee...",an unlikely friendship,Harper Lee,a haik


In [ ]:
hard_poems_compiled_df

,Poem Prompt,Poem,Topic,Style,Structure
0,"Write me a poem about nature's revolt, in the ...",Through silent pines a untamed ghost progresse...,nature's revolt,Robert Frost,a sonne
1,Write me a poem about a child's pure innocence...,Streams babbling in bright spring diversion\nS...,a child's pure innocence,William Wordsworth,a quatrai
2,"Write me a poem about lust, in the style of E....","Moon-mists seduce springs,\nSilent petal frenz...",lust,E.E. Cummings,a haik
3,"Write me a poem about loss, in the style of Em...",In Quiet Groves where Hopelight Wayward Fades\...,loss,Emily Dickinson,a villanell
4,"Write me a poem about war, in the style of Wil...","Shrouded skies, heavy as forebforces nimmrelli...",war,Wilfred Owen,a pantou
...,...,...,...,...,...
74,Write me a poem about the purity of a baby's l...,"Who'd wish more few, golden bubbles in Emerald...",the purity of a baby's laughter,Shel Silverstein,a quatrai
75,Write me a poem about the stillness of a grave...,Beneath shadowy harebell posts silent charcoal...,the stillness of a graveyard,Edgar Allan Poe,a eleg
76,Write me a poem about the secret life of a ros...,"Unheard hums in rosy depths unveil,\nEpoch ble...",the secret life of a rose,William Blake,a sonne
77,"Write me a poem about the end of the world, in...",Petals swept by hollow breath of rain-child ro...,the end of the world,T.S Eliot,a free vers


In [ ]:
from google.colab import files
easy_poems_compiled_df.to_csv('easy_poems.csv')
files.download('easy_poems.csv')
medium_poems_compiled_df.to_csv('medium_poems.csv')
files.download('medium_poems.csv')
hard_poems_compiled_df.to_csv('hard_poems.csv')
files.download('hard_poems.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

GENERATE data for embeddings

In [ ]:
# get emotion pairs with few shot prompting
def generate_opposites():
    prompt = "Create a list of 10 pairs of opposite adjectives that describes poems in the form of [happy/sad, intense/relaxed, formal/informal]. Stick to this format!!"
    messages = [
        {"role": "system", "content": prompt}
    ]

    client = OpenAI(base_url="https://oai.hconeai.com/v1", api_key=os.environ['HELICONE_API_KEY'])
    response = client.chat.completions.create(
      model='gpt-4',
      messages=messages,
      temperature=1,
      max_tokens=8000,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    gpt_response = response.choices[0].message.content
    gpt_response = gpt_response[1:-1]
    # split the response into a list of pairs, stripping spaces
    pairs = gpt_response.split(',')
    # split each pair into a tuple,
    pairs = [[word.strip() for word in pair.split('/')] for pair in pairs]
    return pairs


In [ ]:
descriptions_list = generate_opposites()
print(descriptions_list)

[['expressive', 'restrained'], ['passionate', 'cold'], ['imaginative', 'uncreative'], ['evocative', 'monotonous'], ['lyrical', 'flat'], ['profound', 'superficial'], ['emotional', 'unemotional'], ['structured', 'loose'], ['optimistic', 'pessimistic'], ['beautiful', 'ugly']]


In [ ]:
descriptions_list = [
    ['formal', 'informal'],
    ['traditional', 'modern'],
    ['serious', 'funny'],
    ['romantic', 'cynical'],
    ['rhythmic', 'free'],
    ['intense', 'relaxed'],
    ['emotional', 'rational'],
    ['profound', 'superficial'],
    ['expressive', 'restrained'],
    ['happy', 'sad']
]

'''
,
    ['formal', 'informal'],
    ['simple', 'complex'],
    ['traditional', 'modern'],
    ['emotional', 'rational'],
    ['rhymed', 'unrhymed'],
    ['clear', 'ambiguous'],
    ['positive', 'negative'],
    ['intense', 'relaxed'],
    ['serious', 'funny']
'''

"\n,\n    ['formal', 'informal'],\n    ['simple', 'complex'],\n    ['traditional', 'modern'],\n    ['emotional', 'rational'],\n    ['rhymed', 'unrhymed'],\n    ['clear', 'ambiguous'],\n    ['positive', 'negative'],\n    ['intense', 'relaxed'],\n    ['serious', 'funny']\n"

In [ ]:
# Read Poem CSV data
easy_df = pd.read_csv('/content/drive/MyDrive/easy_poems.csv')

In [ ]:
print(easy_df.columns)

Index(['Unnamed: 0', 'Poem Prompt', 'Poem', 'Topic', 'Style', 'Structure'], dtype='object')


In [ ]:
def generate_baseline_poems(poem_df_row, descriptions_list):
    df = pd.DataFrame(columns=['index', 'adjective', 'poem'])
    og_prompt = poem_df_row['Poem Prompt']
    index = poem_df_row['Unnamed: 0']
    for pair in descriptions_list:
        for word in pair:
            prompt = f"{og_prompt}. Limit the poem to be 40 words. Make sure that the poem is {word}."
            messages = [
                {"role": "system", "content": prompt}
            ]
            client = OpenAI(base_url="https://oai.hconeai.com/v1", api_key=os.environ['HELICONE_API_KEY'])
            response = client.chat.completions.create(
              model='gpt-4',
              messages=messages,
              temperature=1,
              max_tokens=8000,
              top_p=1.0,
              frequency_penalty=0,
              presence_penalty=0
            )
            gpt_response = response.choices[0].message.content
            # print(gpt_response)
            # add index, word, and poem to the baseline_df
            new_row = pd.DataFrame({'index': [index], 'adjective': [word], 'poem': [gpt_response]})
            df = pd.concat([df, new_row], ignore_index=True)
    return df

In [ ]:
# Run data collection in parallel
from multiprocessing import Pool

def process_row(row):
    print(row['Unnamed: 0'])
    df = generate_baseline_poems(row, descriptions_list)
    return df

if __name__ == '__main__':
    baseline_df = pd.DataFrame(columns=['index', 'adjective', 'poem'])

    # Create a multiprocessing Pool
    pool = Pool()

    # Use pool.map to run the function in parallel
    results = pool.map(process_row, [row for _, row in easy_df.iterrows()])

    # Concatenate the results
    baseline_df = pd.concat(results, ignore_index=True)

print(baseline_df.head())

09

10
1
11
2
3
12
13
4
14
5
15
6
16
7
17
8
18
27
28
19
29
20
30
21
31
22
32
23
33
24
25
34
26
35
36
45
46
37
47
38
48
39
40
49
41
50
51
42
43
52
44
53
54
63
55
64
56
57
65
58
66
59
67
60
68
61
69
62
70
  index    adjective                                               poem
0     0       formal  Beneath dawn's cloak emerges the majestic sun,...
1     0     informal  In dawns’ embracing arms of light, we rise,\nE...
2     0  traditional  Sol's velvet touch at dawn's first lie,\nBegui...
3     0       modern  Radiant glow, morning's light doth unveil,\nHa...
4     0      serious  Golden dawn asserts its worth, bold and bright...


In [ ]:
# Original for-loop version
baseline_df = pd.DataFrame(columns=['index', 'adjective', 'poem'])
for index, row in easy_df.iterrows():
  print(index)
  df = generate_baseline_poems(row, descriptions_list)
  baseline_df = pd.concat([baseline_df, df], ignore_index=True)
print(baseline_df.head())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
  index adjective                                               poem
0     0     happy  In that blessed morning's radiant gleam,\nBeyo...
1     0       sad  In tender sorrow, light awakes and cries,\nSwe...
2     1     happy  First love unveiled, in a heart's night bloom,...
3     1       sad  In the rose garden of love, first bloom stirre...
4     2     happy  In silent fields, where cannons shook,\nPeace ...


In [ ]:
from google.colab import files
baseline_df.to_csv('easy_poems_baseline.csv')
files.download('easy_poems_baseline.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>